In [2]:
fege

NameError: name 'fege' is not defined

In [1]:
import sys
sys.path.append('../_amigocloud')

from sqlalchemy import create_engine, text
from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json
import pandas as pd

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [2]:
sys.path.append('..')

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [4]:
ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

In [11]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def cargar_link_to_db(url, cite):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.resumen_control_bio SET link = :url WHERE cite = :cite
            """)
            conn.execute(query, {"url": url, "cite": cite})
            print(f"✔️ Se registro link para CITE: {cite}" )
    except Exception as e:
        print(f"❌ Error al actualizar CITE: {cite}, {e}")

def get_registros_sin_enviar():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.resumen_control_bio
            WHERE enviado = false
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return pd.DataFrame()
    return None

In [6]:
convertir_docx_to_pdf()

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\10697_SLCB_03-09-2025_PAZ REA JAIME EDUARDO_CB-2025-47.docx
✅ Convertido: 10697_SLCB_03-09-2025_PAZ REA JAIME EDUARDO_CB-2025-47.docx
🗑️ Eliminado original: 10697_SLCB_03-09-2025_PAZ REA JAIME EDUARDO_CB-2025-47.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\10739_SLCB_03-09-2025_PERALES VARGAS IVAR JUAN_CB-2025-55.docx
✅ Convertido: 10739_SLCB_03-09-2025_PERALES VARGAS IVAR JUAN_CB-2025-55.docx
🗑️ Eliminado original: 10739_SLCB_03-09-2025_PERALES VARGAS IVAR JUAN_CB-2025-55.docx


  0%|          | 0/1 [00:00<?, ?it/s]

...

C:\Users\Usuario\Desktop\PROYECTOS\_informes\41819_SLCB_03-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-53.docx
✅ Convertido: 41819_SLCB_03-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-53.docx
🗑️ Eliminado original: 41819_SLCB_03-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-53.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\41819_SLCB_04-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-52.docx
✅ Convertido: 41819_SLCB_04-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-52.docx
🗑️ Eliminado original: 41819_SLCB_04-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-52.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\42231_SLCB_03-09-2025_PAZ FERNANDEZ RODRIGO_CB-2025-48.docx
✅ Convertido: 42231_SLCB_03-09-2025_PAZ FERNANDEZ RODRIGO_CB-2025-48.docx
🗑️ Eliminado original: 42231_SLCB_03-09-2025_PAZ FERNANDEZ RODRIGO_CB-2025-48.docx


In [7]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [8]:
contenido = os.listdir(directorio_destino)
contenido

['10697_SLCB_03-09-2025_PAZ REA JAIME EDUARDO_CB-2025-47.pdf',
 '10739_SLCB_03-09-2025_PERALES VARGAS IVAR JUAN_CB-2025-55.pdf',
 '10863_SLCB_03-09-2025_PAZ REA ROSENDO_CB-2025-49.pdf',
 '11860_SLCB_02-09-2025_ROCA SERRANO ARMANDO_CB-2025-50.pdf',
 '15085_SLCB_01-09-2025_CRUZ RUTH JAQUELIN B. DE_CB-2025-56.pdf',
 '15085_SLCB_01-09-2025_CRUZ RUTH JAQUELIN B. DE_CB-2025-58.pdf',
 '1580_SLCB_04-09-2025_CALLE DELGADO IVER_CB-2025-60.pdf',
 '2505_SLCB_01-09-2025_CRUZ BARRIENTOS CRISTIAN F._CB-2025-57.pdf',
 '31088_SLCB_02-09-2025_COLQUE QUIROZ JENNY FELICIDAD_CB-2025-54.pdf',
 '41533_SLCB_02-09-2025_ARNEZ COLQUE ROSARIO ESTELA_CB-2025-51.pdf',
 '41819_SLCB_02-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-59.pdf',
 '41819_SLCB_03-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-53.pdf',
 '41819_SLCB_04-09-2025_AGROPECUARIA GUSTAVITO SRL_CB-2025-52.pdf',
 '42231_SLCB_03-09-2025_PAZ FERNANDEZ RODRIGO_CB-2025-48.pdf']

In [9]:
for file in contenido:
    partes = file.split('_')
    
    cite = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file
    id_drive = '1sEfhWGATb7tkKgr6gcknH3ZA_ed_hGTq'
    path_file = os.path.join(directorio_destino, file)
    print(cite)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    cargar_link_to_db(url_drive, cite)
    if os.path.exists(path_file):
        #os.remove(path_file)
        print("✅ Archivo eliminado.")
    else:
        print("⚠️ El archivo no existe.")

CB-2025-47
[INFO] El archivo '10697_SLCB_03-09-2025_PAZ REA JAIME EDUARDO_CB-2025-47.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '10697_SLCB_03-09-2025_PAZ REA JAIME EDUARDO_CB-2025-47.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-47
✅ Archivo eliminado.
CB-2025-55
[INFO] El archivo '10739_SLCB_03-09-2025_PERALES VARGAS IVAR JUAN_CB-2025-55.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '10739_SLCB_03-09-2025_PERALES VARGAS IVAR JUAN_CB-2025-55.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-55
✅ Archivo eliminado.
CB-2025-49
[INFO] El archivo '10863_SLCB_03-09-2025_PAZ REA ROSENDO_CB-2025-49.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '10863_SLCB_03-09-2025_PAZ REA ROSENDO_CB-2025-49.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-49
✅ Archivo eliminado.
CB-2025-50
[INFO] El archivo '11860_SLCB_02-09-2025_ROCA SERRANO ARMANDO_CB-2025-50.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '11860_SLCB_02-09-2025_ROCA SERR

In [80]:
import psycopg2

In [128]:
def obtener_conexion():
    return psycopg2.connect(
        host='localhost', 
        port=5433, 
        database='utea', 
        user='postgres', 
        password='A123456*')
    
def insertar_mensaje_whatsapp(cod_canero, nombre_canero, numero_contac, mensaje, enviado=False, fecha_enviado=None):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            INSERT INTO notificaciones_wsp.msj_whatsapp (
                cod_canero, nombre_canero, numero_cel, mensaje, enviado, fecha_enviado, numero_contac
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (cod_canero, nombre_canero, 0, mensaje, enviado, fecha_enviado, numero_contac))
        
        conexion.commit()
        cursor.close()
        print("Mensaje insertado correctamente.")
    except Exception as e:
        conexion.rollback()
        print("Error inesperado")
    return None

def marcar_resumen_lib_enviado(id_resumen):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            UPDATE drones_control_bio.resumen_control_bio
            SET enviado = true
            WHERE id = %s
        """, (id_resumen,))
        conexion.commit()
        cursor.close()
        print(f"Mensaje con id={id_resumen} marcado como enviado.")
    except Exception as e:
        conexion.rollback()
        print(f"Error inesperado: {e}")
    finally:
        if conexion:
            conexion.close()
    return None

In [137]:
PATH_XLSX_CONTAC=r'G:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - EQUIPO AVIACION UTEA\\Pulverizacion\\Data\\CONTACTOS.xlsx'

In [162]:
sin_enviar = get_registros_sin_enviar()

In [163]:
cods_ca = list(set(sin_enviar['id']))
cods_ca

[56, 59, 54]

In [164]:
df_contac = pd.read_excel(PATH_XLSX_CONTAC)
df_contac.fillna(0, inplace=True)
df_contac = df_contac[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [165]:
df_contac.head(3)

,cod_ca,nom_ca,telf01,telf02,telf03
0,0,CAÑERO TEST,78194371,0.0,0.0
1,75,AGUILERA TARADELLES NELSON MARIANO,76344582,77054999.0,0.0
2,86,AGUILERA TARADELLES JOSE LUIS,67759316,77010520.0,71384113.0


In [166]:
sin_enviar.head(3)

,id,fecha,institucion,cod_canero,nombre_canero,hectareas,cite,cod_propiedad,propiedad,producto,link,agrocittca,enviado
0,59,2025-09-02,304,31088,COLQUE QUIROZ JENNY FELICIDAD,7.59,CB-2025-54,342,LA PRIMAVERA--PEDRO ARNEZ,['TRICHOGRAMMA'],https://drive.google.com/file/d/1Hw-FsBwHwbfzl...,False,False
1,56,2025-09-02,304,41533,ARNEZ COLQUE ROSARIO ESTELA,12.74,CB-2025-51,172,EL TESORO--ARNEZ Y COLQUE,['TRICHOGRAMMA'],https://drive.google.com/file/d/11cr24IyIIlu2s...,False,False
2,54,2025-09-03,69,10863,PAZ REA ROSENDO,38.15,CB-2025-49,47,SANTA MARTHA,['TRICHOGRAMMA'],https://drive.google.com/file/d/1gAq9OWGSrkoFU...,False,False


In [168]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'Codigo: {cod} no tinene numeros validos registrados')
            continue
            
        for num in telefonos_validos:
            msj = f"Saludos *{row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRA* para hacer llegar una copia del informe de servicio de control biologico en caña de azucar. Adjunto informe: {row['link']}"
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 31088 - COLQUE QUIROZ JENNY FELICIDAD no tiene registro telefonico
Codigo: 41533 - ARNEZ COLQUE ROSARIO ESTELA no tiene registro telefonico
Codigo: 10863 - PAZ REA ROSENDO no tiene registro telefonico
